[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jolin-io/KI2022-tutorial-universal-differential-equations/main?filepath=03%20deep%20dive%20into%20universal%20differential%20equations.ipynb)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.1-darkmode.webp">
</a>

# Deep dive into Universal Differential Equations in <img height="60px" style='height:60px;display:inline;' alt="Julia" src="https://julialang.org/assets/infra/logo.svg">

In [ ]:
import DifferentialEquations, SciMLSensitivity, DiffEqFlux
import Symbolics, ModelingToolkit, DataDrivenDiffEq
import Optimization, OptimizationOptimisers, OptimizationOptimJL
import Lux, ComponentArrays
import Plots, Random, Statistics, StatsBase, DelimitedFiles

using CommonSolve: solve

rng = Random.default_rng()
Random.seed!(rng, 12345)

Outline of this extensive deep dive:
1. Scientific Machine Learning with UDEs
    1. Differential Equations
    2. DiffEq within Machine Learning
    3. Machine Learning within DiffEq
    4. Machine Learning within DiffEq - alternative perspective
    5. More UDEs
2. Symbolic Regression with DataDrivenDiffEq
    1. Symbolic regression without UDE
    2. Symbolic regression with UDE

# Scientific Machine Learning with UDEs

The term Universal Differential Equations was introduced in the paper [Universal Differential Equations for Scientific
Machine Learning by Rackauckas et. al. 2021](https://arxiv.org/pdf/2001.04385.pdf)

**UDE is about using machine learning as part of differential equation problems.** As such it is one way of combining scientific model-based approaches with machine learning techniques, which is often named scientific machine learning. 

Another combination of machine learning and differential equations are for example physics-informed neural networks (PINN). These are not the topic of today, but have a look at [NeuralPDE.jl](https://github.com/SciML/NeuralPDE.jl) if you are interested.

Here an overview over the scientific machine learning ecosystem as described in the UDE paper:
![](./assets/overview_sciml_ecosystem.png)

This is a huge ecosystem. For today we focuse mostly on the last layer of implementing Differential Equations which depend on Neural Networks directly.

The following code is a compilation and update from [official UDE paper example](https://github.com/ChrisRackauckas/universal_differential_equations/blob/master/LotkaVolterra).

# Let's start with some real data

The data has been taken from https://jmahaffy.sdsu.edu/courses/f00/math122/labs/labj/q3v1.htm
(Originally published in E. P. Odum (1953), Fundamentals of Ecology, Philadelphia, W. B. Saunders)


In [ ]:
hudson_bay_data = DelimitedFiles.readdlm("assets/hudson_bay_data.dat", '\t', Float32, '\n')

In [ ]:
# normalize time to start at 0
t = hudson_bay_data[:, 1] .- hudson_bay_data[1, 1]
tspan = (t[begin], t[end])

# Measurements of prey and predator
X = Matrix(transpose(hudson_bay_data[:, 2:3]))
# Normalize the data; since the data domain is strictly positive
# we just need to divide by the maximum
xscale = maximum(X, dims =2)
X .= 1f0 ./ xscale .* X

# Plot the data
Plots.scatter(t, X', xlabel = "t", ylabel = "x(t), y(t)")
Plots.plot!(t, X', xlabel = "t", ylabel = "x(t), y(t)")

## DifferentialEquations.jl

We can model this data using the [Lotka-Volterra equations](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations): Population of rabbits and foxes

<center>

rabbits: $ x^\prime = \alpha x - \beta x y $

</center>

the rate of change of the prey's population is given by its own growth rate ($\alpha$) minus the rate at which it is preyed upon ($\beta$).


<center>

foxes: $ y^\prime = \gamma x y - \delta y $

</center>

the rate of change of the predator's population depends upon the rate at which it consumes prey ($\gamma$), minus its intrinsic death rate ($\delta$)

In [ ]:
function lotka_volterra(du, u, p, t)
    x, y = u
    α, β, δ, γ = p
    du[1] = dx = α*x - β*x*y
    du[2] = dy = -δ*y + γ*x*y
end
u0 = X[:, 1]
tspan = (0.0, 20.0)
p = [0.6, 0.8, 0.6, 0.8]
ode_prob = DifferentialEquations.ODEProblem(lotka_volterra, u0, tspan, p)

In [ ]:
Plots.plot(solve(ode_prob, saveat=t))
Plots.scatter!(t, X', xlabel = "t", ylabel = "x(t), y(t)")

let's fit the data

## DiffEq within Machine Learning

This just means we learn the DiffEq parameters via gradient-based Optimization.

In [ ]:
function predict(parameters, ode_prob=ode_prob, t=t)
    solve(ode_prob, saveat = t, p = parameters)
end
function loss_function(parameters, data)
    pred = Array(predict(parameters))
    return sum(abs2, pred .- data)
end

In [ ]:
ps_initial = ode_prob.p
Plots.plot(predict(ps_initial))
Plots.scatter!(t, X', xlabel = "t", ylabel = "x(t), y(t)")

In [ ]:
loss_function(ps_initial, X)

In [ ]:
losses = Float64[]
function callback(p, l)
    push!(losses, l)
    if length(losses) % 50 == 0
        Plots.plot(losses, show = :inline, yscale = :log10,
            label = "loss", xlabel = "#epochs", ylabel="loss (log10 scale)")
    end
    return false  # return bool `halt`
end

In [ ]:
minimizer = ps_initial

opt_function = Optimization.OptimizationFunction(
    (ps, data) -> loss_function(ps, data),
    Optimization.AutoZygote(),
)

for (optimizer, maxiters) = [
        (OptimizationOptimisers.Adam(0.1), 300),
        (OptimizationOptimisers.Adam(0.01), 500),
    ]
    opt_prob = Optimization.OptimizationProblem(opt_function, minimizer, X)
    opt_sol = solve(opt_prob, optimizer, callback = callback, maxiters = maxiters)
    minimizer = opt_sol.minimizer
end

ps_trained = minimizer

In [ ]:
Plots.plot(predict(ps_trained))
Plots.scatter!(t, transpose(X), xlabel = "t", ylabel = "x(t), y(t)")

👉 try different initial parameter, make the problem harder

👉 experiment with the optimizers [Adam](https://fluxml.ai/Flux.jl/stable/training/optimisers/#Flux.Optimise.Adam) and try different configurations

In [ ]:
# your space

## Machine Learning within DiffEq

We can use machine learning within Differential Equations to leave parts of our model as black-box which are going to be learned.

For simplicity we generate data from some fully known model

In [ ]:
ps_ideal = ps_trained
ode_prob_ideal = DifferentialEquations.ODEProblem(lotka_volterra, u0, tspan, ps_ideal)
ode_sol_ideal = solve(ode_prob_ideal, saveat = 0.1)

# Ideal data
X_ideal = Array(ode_sol_ideal)
t = ode_sol_ideal.t

noise_magnitude = 5e-2
X = X_ideal
X = X .+ (noise_magnitude*Statistics.mean(X, dims=2)) .* randn(eltype(X), size(X))

Plots.plot(ode_sol_ideal)
Plots.plot!(t, X')

### The machine learning part

In [ ]:
# Gaussian RBF as activation
rbf(x) = exp.(-(x.^2))

# Define the network 2->5->5->5->2
model_lux = Lux.Chain(
    Lux.Dense(2,5,rbf),
    Lux.Dense(5,5, rbf),
    Lux.Dense(5,5, tanh),
    Lux.Dense(5,2)
)

In [ ]:
ps_lux, st_lux = Lux.setup(rng, model_lux)

### Bringing ml into the differential equations

We model the linear terms explicitly (like we are sure that linear terms are apt for our scenario), but leave the interaction-terms to our neural black-box.

Let's see whether we can reconstruct the interactions.

In [ ]:
# Define the hybrid model
function ude_dynamics!(du,u, p, t)
    u_pred, _st_lux = model_lux(u, p.ps_lux, st_lux) # Network prediction
    # We assume a linear birth rate for the prey
    du[1] = p.ps_ode[1]*u[1] + u_pred[1]
    # We assume a linear decay rate for the predator
    du[2] = -p.ps_ode[2]*u[2] + u_pred[2]
end

# Get the initial parameters, first two are linear birth/decay of prey and predator
ps_initial = ComponentArrays.ComponentVector((
    ps_ode = rand(rng, Float32, 2),
    ps_lux = ps_lux,
))
u0 = X[:, 1]
ode_prob_nn = DifferentialEquations.ODEProblem(ude_dynamics!, u0, tspan, ps_initial)

In [ ]:
function predict(parameters, t=t, ode_prob=ode_prob_nn)
    solve(
        ode_prob,
        p = parameters,
        saveat = t,
        # sensealg = SciMLSensitivity.ForwardDiffSensitivity()
    )
end

👉 plot our initial guess for the ode solution (given the initial parameters)

In [ ]:
# your space

### Training

Training is a bit more elaborate. We first use a special training loss provided by `DiffEqFlux`. It is called `muliple_shoot` which essentially devides the training data into pieces and learns on the single pieces instead of learning everything at once.

For more details on `multiple_shoot` see the [DiffEqFlux.jl documentation](https://diffeqflux.sciml.ai/stable/examples/multiple_shooting/).

In [ ]:
function shooting_loss(parameters, X=X, t=t, ode_prob=ode_prob_nn,
        group_size=5, continuity_term=200.0f0)
    
    loss_compare(data, pred) = sum(abs2, data - pred)
    
    loss, pred = DiffEqFlux.multiple_shoot(
        parameters, X, t, ode_prob, loss_compare, DifferentialEquations.Tsit5(), group_size;
        continuity_term = continuity_term)
    loss
end

We also define a standard loss over the full data. The loss comes with an extra penalty which forces parameters to be small.

In [ ]:
function loss(parameters, X=X)
    _X_pred = predict(parameters)
    _X_pred.retcode == :Success || return Inf
    X_pred = Array(_X_pred)
    loss_diff = sum(abs2, X - X_pred) / size(X, 2)
    
    loss_penalty = sum(abs2, parameters.ps_lux) / length(parameters.ps_lux)
    factor_penalty = convert(eltype(parameters), 1e-4)
    loss_diff + factor_penalty * loss_penalty 
end

👉 run both losses

In [ ]:
# your space

Let's train

In [ ]:
# Container to track the losses
losses = Float32[]

# Callback to show the loss during training
callback(parameters, args...) = begin
    l = loss(parameters) # Equivalent L2 loss
    push!(losses, l)
    if length(losses) % 50 == 0
        Plots.plot(losses, show = :inline, yscale = :log10,
            label = "loss", xlabel = "#epochs", ylabel="loss (log10 scale)")
    end
    return false  # return bool `halt`
end

we train with different optimizers and loss functions

In [ ]:
losses = Float32[]
minimizer = ps_initial

for (opt_alg, maxiters, loss_func) = [
        (OptimizationOptimisers.Adam(0.01), 200, shooting_loss)
        (OptimizationOptimisers.Adam(0.01), 100, loss)
    ]
    opt_func = Optimization.OptimizationFunction((ps, _) -> loss_func(ps), Optimization.AutoZygote())   
    opt_prob = Optimization.OptimizationProblem(opt_func, minimizer) 
    opt_sol = solve(opt_prob, opt_alg, maxiters = maxiters, callback = callback)
    minimizer = opt_sol.minimizer
end
ps_trained = minimizer

Did it work out?

In [ ]:
X_pred = Array(predict(ps_trained))

# Trained on noisy data vs real solution
p1 = Plots.scatter(t, X[1,:], label="Data", alpha=0.2, xlabel = "t", ylabel = "u1(t)")
Plots.plot!(t, X_ideal[1,:], label="ideal")
Plots.plot!(t, X_pred[1,:], label="UDE Approximation")

p2 = Plots.scatter(t, X[2,:], label="Data", alpha=0.2, xlabel = "t", ylabel = "u2(t)")
Plots.plot!(t, X_ideal[2,:], label="ideal")
Plots.plot!(t, X_pred[2,:], label = "UDE Approximation")

Plots.plot(p1, p2, layout=(2,1))

Oh! We need to improve.

👉 adapt the training procedure(the number of iterations, the [Adam](https://fluxml.ai/Flux.jl/stable/training/optimisers/#Flux.Optimise.Adam) config, ...) to make our model fit the data at least reasonable

In [ ]:
@show ps_trained.ps_ode
@show ode_prob_ideal.p[[1, 3]];

👉 fix one of the linear factors to the true value and see whether we can now infer the other correctly

### Simulating the future

👉 now that the training looks good, let's check whether the model is stable on the long run

simulate our `nn_ode_prob` for some time into the future (hint: you may want to change `tspan`)

In [ ]:
# your space

##  Machine Learning within DiffEq - alternative perspective 

The famous paper **Neural Ordinary Differential Equations (Chen et al. 2019)** introduced the following intuition for Neural Ordinary Differential Equations.

Residual Neural Network (discrete difference layers)
$$h_{t+1} = h_t + f(h_t, \theta_t)$$

Neural Ordinary Differential Equations
$$\frac{dh(t)}{dt} = f(h(t), t, \theta)$$

![](https://www.jolin.io/assets/examples/NeuralODE-Comparing-ResNet.png)

## More UDEs

One key aspect of Julia's scientific machine learning stack is the immense features it provides.

Just a short summary from the UDE paper.

![UDE features](./assets/ude_overview_features.png)

and here the benchmarks
![UDE benchmarks](./assets/ude_benchmarks.png)

# Symbolic regression

Symbolic regressions is the discipline of fitting mathematical formular to given data. We use DataDrivenDiffEq.jl

First, generate the basis functions, multivariate polynomials up to deg 5 and sine


In [ ]:
Symbolics.@variables u[1:2]
b = DataDrivenDiffEq.polynomial_basis(u, 5)
basis = DataDrivenDiffEq.Basis(b, u)

## Symbolic regression without UDE

for all symbolic regression we use the same solver

In [ ]:
# Create the thresholds which should be used in the search process
λ = Float32.(exp10.(-7:0.1:5))
# Create an optimizer for the SINDy problem
opt = DataDrivenDiffEq.STLSQ(λ)

symbolic regression from true derivate data works perfectly

In [ ]:
DX = Array(ode_sol_ideal(t, Val{1}))
ddd_prob_fullideal = DataDrivenDiffEq.DataDrivenProblem(X, t = t, DX = DX)

ddd_sol_fullideal = solve(ddd_prob_fullideal, basis, opt,
    maxiter = 10_000, progress = true, denoise = true, normalize = true)

println(ddd_sol_fullideal)
println(DataDrivenDiffEq.result(ddd_sol_fullideal))
println()
println(DataDrivenDiffEq.parameter_map(ddd_sol_fullideal))

Symbolic regression from approx derivatives via collocation

In [ ]:
# Create the problem using a gaussian kernel for collocation
ddd_prob_fullreal = DataDrivenDiffEq.ContinuousDataDrivenProblem(X, t, DataDrivenDiffEq.GaussianKernel())
ddd_sol_fullreal = solve(ddd_prob_fullreal, basis, opt,
    maxiter = 10_000, progress = true, denoise = true, normalize = true)

println(ddd_sol_fullreal)
println(DataDrivenDiffEq.result(ddd_sol_fullreal))
println()
println(DataDrivenDiffEq.parameter_map(ddd_sol_fullreal))

we compare the prediction of the derivative

## Symbolic regression with UDE

We want to apply symbolic regression to the neural network part.

Importantly, the neural net only captured the **interactions** between predators and prey.
The **linear parts** were already given (structurely), and fit separately -  they don't matter here.

Above we used `DataDrivenDiffEq` to approximate a derivative.
We can actually use the same package to appxorimate direct functions.

In [ ]:
# Neural network guess
Y_nn, _st_lux = model_lux(X_pred, ps_trained.ps_lux, st_lux)

ddd_prob_nn = DataDrivenDiffEq.DataDrivenProblem(X_pred, Y=Y_nn)
ddd_sol_nn = solve(ddd_prob_nn, basis, opt, maxiter = 10_000, progress = true, normalize = false, denoise = true)

println(ddd_sol_nn)
println(DataDrivenDiffEq.result(ddd_sol_nn))
println()
println(DataDrivenDiffEq.parameter_map(ddd_sol_nn))

This looks kind of okay

To check, that the symbolic regression `X~Y` really works, we can apply it to the ideal data

In [ ]:
Y_ideal = [
    -ps_ideal[2] * (X_ideal[1,:] .* X_ideal[2,:])'
    ps_ideal[4] * (X_ideal[1,:] .* X_ideal[2,:])'
]

👉 apply `DataDrivenProblem` to ideal case

# That was the deep-dive into Universal Differential Equations in julia - Thank you for participating 🙂

I've prepared a **bonus topic** about combining differential equations with bayesian inference, i.e. probabilistic parameter and error estimation: [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jolin-io/KI2022-tutorial-universal-differential-equations/main?filepath=04%20introduction%20to%20bayesian%20differential%20equations.ipynb)

If you have question, suggestions, or you are just interested in Julia, contact me:
- Stephan Sahm stephan.sahm@jolin.io

### Further Material

- [Blog Post DiffEqFlux.jl](https://julialang.org/blog/2019/01/fluxdiffeq/)
- [Documentation DiffEqFlux.jl](https://diffeqflux.sciml.ai/stable/)
- [Paper Neural Ordinary Differential Equations (Chen et al. 2019)](https://arxiv.org/abs/1806.07366)
- [Paper Universal Differential Equations for SciML (Rackauckas et al. 2020)](https://arxiv.org/abs/2001.04385)
- [Documentation DataDrivenDiffEq.jl](https://datadriven.sciml.ai/stable), [linear ODE example](https://datadriven.sciml.ai/stable/examples/2_linear_continuous_system/), [nonlinear ODE example](https://datadriven.sciml.ai/stable/examples/4_nonlinear_continuous_system/)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.1-darkmode.webp">
</a>

#### Supported by [Jolin.io](https://www.jolin.io)

Jolin.io is an IT-consultancy for high-performance computing and data science

We are there to help you, if you want to
- try out Julia at your company, or
- transition Matlab, Fortran, R, Python, etc. to Julia
- or speed up your existing code